# Stacks Maker Section

In [10]:
import random
import parcels

In [11]:
''' split into two arrays :
    parcels_count (
        ssp : 29
        smp : 22
        hssp : 30
    ), and weight dict (
        ssp : [1, 2, 8, 1, 2, 3, 4, 7]
    ) '''
class Stack:
    def __init__(self, parcel_members, weight_each, stack_height, stack_width, stack_length, stack_name):
        self.members = parcel_members
        self.weight_each = weight_each
        self.height = stack_height
        self.width = stack_width
        self.length = stack_length
        self.name = stack_name
        self.area = self.width * self.length
        self.volume = self.area * self.height
        
class StacksMaker:
    same_type_stacks_amount = None
    mix_type_stacks_amount = None
    stack_zero = Stack([], weight_each={
        'fl_1' : [0]
    }, stack_height=0, stack_width=0, stack_length=0, stack_name=0)
    def __init__(self, parcels, parcels_weight):
        self.parcels = parcels
        self.parcels_weight = parcels_weight
    def make_stacks(self):
        same_type_stacks = {
            'ssp' : 0, 
            'smp' : 0,
            'hssp' : 0, 
            'mlp' : 0, 
            'mp' : 0, 
            'lp' : 0, 
            'lhp' : 0,
            'etc' : 0
        }
        mix_type_stacks = {
            201 : 0, 
            202 : 0, 
            203 : 0, 
            204 : 0, 
            205 : 0, 
            206 : 0, 
            207 : 0, 
            208 : 0
        }
        value_mod_same_type = [2, 3, 4, 2, 3, 3]
        # making same-type stacking (101 - 106)
        idx = 0 
        for stack_name, value in self.parcels.items():
            if idx >= len(value_mod_same_type):
                break
            stack_count = (value // value_mod_same_type[idx])
            same_type_stacks[stack_name] += stack_count
            self.parcels[stack_name] -= (stack_count * value_mod_same_type[idx])
            idx += 1
        while True:
            if self.parcels['smp'] >= 2 and self.parcels['hssp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['smp'] -= 2
                self.parcels['hssp'] -= 1
            elif self.parcels['smp'] >= 2 and self.parcels['etc'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['smp'] -= 2
                self.parcels['etc'] -= 1
            elif self.parcels['ssp'] >= 2 and self.parcels['etc'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['ssp'] -= 2
                self.parcels['etc'] -= 1
            elif self.parcels['etc'] >= 2 and self.parcels['lhp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['etc'] -= 2
                self.parcels['lhp'] -= 1
            elif self.parcels['hssp'] >= 3 and self.parcels['mp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['hssp'] -= 3
                self.parcels['mp'] -= 1
            elif self.parcels['smp'] >= 6 and self.parcels['mlp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['smp'] -= 6
                self.parcels['mlp'] -= 1
            elif self.parcels['lp'] >= 2 and self.parcels['lhp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['lp'] -= 2
                self.parcels['lhp'] -= 1
            elif self.parcels['smp'] >= 6 and self.parcels['mp'] >= 1:
                mix_type_stacks[201] += 1
                self.parcels['smp'] -= 6
                self.parcels['mp'] -= 1
            else :
                break
        same_type_stacks_fixed = {
            101 : same_type_stacks['hssp'],
            102 : same_type_stacks['smp'], 
            103 : same_type_stacks['ssp'], 
            104 : same_type_stacks['mp'], 
            105 : same_type_stacks['mlp'], 
            106 : same_type_stacks['lp']
        }
        StacksMaker.same_type_stacks_amount = same_type_stacks_fixed
        StacksMaker.mix_type_stacks_amount = mix_type_stacks
        return same_type_stacks_fixed, mix_type_stacks, self.parcels
    def make_objects(self, same_type_stacks, mix_type_stacks):
        stacks = []
        stacks_combined = {**same_type_stacks, **mix_type_stacks}
        for name, value in stacks_combined.items():
            stack_info = parcels.Info.get_stack_info(name)
            stack_members = parcels.Info.get_stack_members(name)
            for _ in range(value):
                stack_obj = Stack(
                    stack_name=name, 
                    stack_height=stack_info[2], 
                    stack_width=stack_info[0], stack_length=stack_info[1], parcel_members=stack_members, weight_each=self.find_weight(name))
                stacks.append(stack_obj)
        return stacks
    def find_weight(self, stack_name):
        weight = {}
        for key, value in parcels.Info.get_stack_members(stack_name).items():
            weight_per_fl = list()
            for x in range(len(value)):
                if value[x] == 'ssp' :
                    min_range = 1 
                    max_range = 50
                elif value[x] == 'smp':
                    min_range = 5 
                    max_range = 50
                elif value[x] == 'hssp':
                    min_range = 12 
                    max_range = 50
                elif value[x] == 'mlp':
                    min_range = 20 
                    max_range = 50
                elif value[x] == 'mp':
                    min_range = 28 
                    max_range = 50
                elif value[x] == 'lp' or value[x] == 'lhp':
                    min_range = 37 
                    max_range = 50
                weight_per_fl.append(random.randint(min_range, max_range))
            weight[key] = weight_per_fl
        return weight
    def execute(self):
        same_type_stacks, mix_type_stacks, remaining_parcels = self.make_stacks()
        stacks_obj = self.make_objects(same_type_stacks, mix_type_stacks)
        stacks_obj.append(StacksMaker.stack_zero)
        return stacks_obj, remaining_parcels, same_type_stacks, mix_type_stacks

In [12]:
import numpy as np
dummy_parcels = {
    'ssp' : 100,
    'smp' : 500, 
    'hssp' : 800, 
    'mlp' : 100, 
    'mp' : 900, 
    'lp' : 300,
    'lhp' : 100,
    'etc' : 70
}
dummy_weights = {
    'ssp' : list(np.random.randint(1, 6, dummy_parcels['ssp'])),
    'smp' : list(np.random.randint(1, 9, dummy_parcels['smp'])),
    'hssp' : list(np.random.randint(1, 15, dummy_parcels['hssp'])),
    'mlp' : list(np.random.randint(1, 19, dummy_parcels['mlp'])),
    'mp' : list(np.random.randint(1, 25, dummy_parcels['mp'])),
    'lp' : list(np.random.randint(1, 60, dummy_parcels['lp'])),
    'lhp' : list(np.random.randint(1, 85, dummy_parcels['lhp'])), 
    'etc' : list(np.random.randint(1, 29, dummy_parcels['etc']))
}

stack_maker = StacksMaker(dummy_parcels, dummy_weights)
stacks, remaining_parcels, same_type_stacking, mix_type_stacking = stack_maker.execute()

In [13]:
mix_type_stacking

{201: 35, 202: 0, 203: 0, 204: 0, 205: 0, 206: 0, 207: 0, 208: 0}

In [14]:
for j in stacks:
    print(j.name)

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102
102


In [15]:
remaining_parcels

{'ssp': 0,
 'smp': 0,
 'hssp': 0,
 'mlp': 0,
 'mp': 0,
 'lp': 0,
 'lhp': 66,
 'etc': 1}

In [24]:
((28.44 - 32.14) / 10) + (12 - 16) + (2.37 - 3.5)

-5.5

# Genetic Algorithm Section
## Population Informations : 
        Length = 100
        Chromosome length = 33
     
##### Weight balance => Balancing when generating initial population

In [16]:
import math
from sklearn import preprocessing
import os
from IPython.display import clear_output
import copy

In [17]:
class GeneticAlgorithm:
    def __init__(self, stacks, container_id):
        self.stacks = stacks
        self.container_id = container_id
        self.container_info = parcels.Info.get_container_info(self.container_id)
    def initial_population_generator(self):
        # creating pop
        population = list()
        for _ in range(100):
            chromosome = list()
            for _ in range(33):
                chromosome.append(random.choice(self.stacks))
            population.append(chromosome)
        #balancing pop
        for index, chrom in enumerate(population):
            for idx, stack in enumerate(chrom):
                if idx == 0:
                    pass
                else :
                    if idx < 19 and (stack.volume < chrom[idx-1].volume):
                        population[index][idx-1], population[index][idx] = population[index][idx], population[index][idx-1]
                    elif stack.volume > chrom[idx-1].volume :
                        population[index][idx-1], population[index][idx] = population[index][idx], population[index][idx-1]
        return population
    def fitness_calculation(self, population, container_id):
        #clear_output(wait=True)
        container_info = parcels.Info.get_container_info(container_id)
        chromosome_scores = list()
        balancing_scores = list()
        sum_area = 0
        for chrom in population:
            weight_balancing = list()
            sum_weight_section = 0
            longest_length = 0
            widest_width = 0
            length = []
            final_score = 0
            for idx, stack in enumerate(chrom):
                sum_area += stack.area
                if idx == 10 or idx == 21 or idx == 32:
                    weight_balancing.append(sum_weight_section)
                    sum_weight_section = 0
                for key, value in stack.weight_each.items():
                    sum_weight_section += sum(value)
                if stack.length > longest_length:
                    longest_length = stack.length
                
                if idx % 3 == 0:
                    length.append(longest_length)
                    longest_length = 0
                    if widest_width == 0:
                        final_score -= 0
                    else :
                        final_score -= 1 - (abs(container_info[0] - widest_width) / container_info[0])
                    widest_width = 0
                if widest_width < stack.width:
                    widest_width = stack.width
            sum_area = 1 - ((abs(((container_info[0] * container_info[1]) - sum_area))) / (container_info[0] * container_info[1]))
            weight_balancing_score = 1 - (abs(((weight_balancing[0] - weight_balancing[1]) + (weight_balancing[1] - weight_balancing[2]))) / 1000)
            length_score = 1 - (abs((self.container_info[1] - sum(length)) / self.container_info[1]))
            final_score += (sum_area + weight_balancing_score + length_score)
            # final_score = sum_area + weight_balancing_score + length_score
            chromosome_scores.append(final_score)
        #normalizing score
        # norm_area = np.linalg.norm(area_scores)
        # norm_length = np.linalg.norm(length_scores)
        # norm_balancing = np.linalg.norm(balancing_scores)
        # # normalized_area = list(area_scores / norm_area)
        # normalized_length = (length_scores - min(length_scores)) / (max(length_scores) - min(length_scores))
        # normalized_balancing = (balancing_scores - min(balancing_scores)) / (max(balancing_scores) - min(balancing_scores))
        # for j in range(len(normalized_balancing)):
        #     chromosome_scores.append(area_scores[j] + normalized_length[j] + normalized_balancing[j])
        # norm = np.linalg.norm(chromosome_scores)
        # normalized_scores = chromosome_scores / norm
        # for j in range(len(chromosome_scores)):
        #     chromosome_scores[j] = (chromosome_scores[j] - min(chromosome_scores)) / (max(chromosome_scores) - min(chromosome_scores))
        return chromosome_scores
    def selection(self, population, fitness_scores):
        selected_chromosomes = list()
        cnt = 0
        for _ in fitness_scores:
            if cnt == 30:
                break
            max_score_idx = fitness_scores.index(max(fitness_scores))
            selected_chromosomes.append(population[max_score_idx])
            fitness_scores[max_score_idx] = -99
            cnt += 1
        return selected_chromosomes
    def crossover(self, offspring_selection):
        offspring_crossover = list()
        uniform_perf_positions = list()
        for _ in range(int(33/5)):
            uniform_perf_positions.append(random.randint(0, 32))
        for x in range(len(offspring_selection)):
            child_one = offspring_selection[x]
            child_two = offspring_selection[x]
            parent_one = x % 2
            parent_two = x % 3
            for j in range(33):
                if j in uniform_perf_positions:
                    child_one[j] = offspring_selection[parent_one][j]
                    child_two[j] = offspring_selection[parent_two][j]
                else :
                    child_one[j] = offspring_selection[parent_two][j]
                    child_two[j] = offspring_selection[parent_one][j]
            offspring_crossover.append(child_one)
            offspring_crossover.append(child_two)
        # for x in range(len(offspring_selection)):
        #     parent_1 = x % 2
        #     parent_2 = x % 3
        #     child = offspring_selection[parent_1][0:17] + offspring_selection[parent_2][17:]
        #     child_2 = offspring_selection[parent_2][0:17] + offspring_selection[parent_1][17:]
        #     offspring_crossover.append(child)
        #     offspring_crossover.append(child_2)
        return offspring_crossover
    def mutation(self, offspring_crossover, stacks):
        ''' Figure out how tf could I be able to put sorta shitty genes whichs stuck in the overwidth usage row
        Method 1: Find the empty row (more than 2 genes in the row are 0) => find a row thats overwidth => pull out the problemed stack
        => put it into the row (the row must be empty or have enough space to store the pulled out stack.'''
        offspring_mutation = list()
        for _ in range(10) :
            mutating_chrom = random.choice(offspring_crossover)
            sections_chrom = [[mutating_chrom[0:2]], [mutating_chrom[3:5]], [mutating_chrom[6:8]], 
            [mutating_chrom[9:11]], [mutating_chrom[12:14]], [mutating_chrom[15:17]], [mutating_chrom[18:20]],
            [mutating_chrom[21:23]], [mutating_chrom[24:26]], [mutating_chrom[27:29]], [mutating_chrom[30:32]]]
            zeros = 0
            indexes = []
            for j in sections_chrom:
                for idx, x in enumerate(j):
                    if zeros == 2:
                        indexes.append(idx)
                    if idx % 3 == 0:
                        zeros = 0
                    if x == StacksMaker.stack_zero:
                        zeros += 1
            for _ in range(3):
                mutating_chrom[random.randint(0, 32)] = copy.deepcopy(random.choice(stacks))
            for _ in range(3):
                rand_pos_one = random.randint(0, 32)
                rand_pos_two = random.randint(0, 32)
                rand_pos_none = 0
                if (rand_pos_one + 1) % 3 == 0 or (rand_pos_one + 2) % 3 == 0 or (rand_pos_one) % 3 == 0:
                    if mutating_chrom[rand_pos_one] != StacksMaker.stack_zero:
                        try :
                            if mutating_chrom[rand_pos_one - 1] != StacksMaker.stack_zero and mutating_chrom[rand_pos_one + 1] != StacksMaker.stack_zero and mutating_chrom[rand_pos_one] != StacksMaker.stack_zero:
                                rand_pos_none = random.randint(rand_pos_one - 1, rand_pos_one + 1)
                            elif mutating_chrom[rand_pos_one] != StacksMaker.stack_zero and mutating_chrom[rand_pos_one + 1] != StacksMaker.stack_zero and mutating_chrom[rand_pos_one + 2] != StacksMaker.stack_zero:
                                rand_pos_none = random.randint(rand_pos_one, rand_pos_one + 2)
                            else :
                                rand_pos_none = random.randint(rand_pos_one - 2, rand_pos_one)
                        except:
                            pass
                if len(indexes) == 0:
                    rand_choice_zero = rand_pos_two
                else :
                    rand_choice_zero = random.choice(indexes)
                mutating_chrom[rand_pos_none], mutating_chrom[rand_choice_zero] = mutating_chrom[rand_choice_zero], mutating_chrom[rand_pos_none]
            offspring_mutation.append(mutating_chrom)
        # for _ in range(10):
        #     mutating_chrom = random.choice(offspring_crossover)
        #     rows = []
        #     row_sub = []
        #     for idx, value in enumerate(mutating_chrom):
        #         if idx % 3 == 0:
        #             rows.append(row_sub)
        #             row_sub.clear()
        #         row_sub.append(value)
        #     row_cnt = 0
        #     sum_row = 0
        #     zeros = 0
        #     overwidth_rows = []
        #     underwidth_rows = []
        #     overwidth_width = []
        #     underwidth_width = []
        #     for row in rows:
        #         for idx, element in enumerate(row):
        #             if element.name == 0:
        #                 zeros += 1
        #             sum_row += element.width
        #             if idx % 3 == 0:
        #                 if sum_row > self.container_info[0]:
        #                     overwidth_rows.append(row_cnt)
        #                     overwidth_width.append(sum_row)
        #                 if zeros >= 2:
        #                     underwidth_rows.append(row_cnt)
        #                     underwidth_width.append(sum_row)
        #                 zeros = 0
        #                 row_cnt += 1
        #                 sum_row = 0
        #     if len(overwidth_rows) == 0:
        #         break
        #     else :
        #         if len(underwidth_rows) == 0:
        #             for j in range(len(overwidth_rows)):
        #                 min_stack = rows[overwidth_rows[j]][0]
        #                 min_area = 0
        #                 for x in rows[overwidth_rows[j]]:
        #                     if x.area > min_area:
        #                         min_area = x.area
        #                         min_stack = x
        #                 index = rows[overwidth_rows[j]].index(min_stack)
        #                 rows[overwidth_rows[j]][index] = copy.deepcopy(StacksMaker.stack_zero)
        #         else :
        #             for j in range(len(underwidth_rows)):
        #                 under_width = random.randint(0, 2)
        #                 over_width_rand = random.choice(overwidth_rows)
        #                 over_width_rand_pos = random.randint(0, 2)

        #                 rows[underwidth_rows[j]][under_width], rows[over_width_rand][over_width_rand_pos] = rows[over_width_rand][over_width_rand_pos], rows[underwidth_rows[j]][under_width]

        #     new_mutate_chromosome = rows[0] + rows[1] + rows[2] + rows[3] + rows[4] + rows[5] + rows[6] + rows[7] + rows[8] + rows[9] + rows[10]
        #     offspring_mutation.append(new_mutate_chromosome)
        # stacks_width = list()
        # for x in stacks:
        #     stacks_width.append(x.width)
        # for _ in range(10):
        #     mutate_chrom = random.choice(offspring_crossover)
        #     for _ in range(3):
        #         rand_pos = random.randint(0, 32)
        #         if mutate_chrom[rand_pos].width > 1.185:
        #             for x in stacks_width:
        #                 if x < 1.185:
        #                     index = stacks_width.index(x)
        #                     mutate_chrom[rand_pos] = stacks[index]
        #     #     else :
        #     #         for x in stacks_width:
        #     #             if x > 1.185:
        #     #                 index = stacks_width.index(x)
        #     #                 mutate_chrom[rand_pos] = stacks[index]
        #     offspring_mutation.append(mutate_chrom)

        # for _ in range(10):
        #     overwidth_arr = list()
        #     width_val = 0
        #     rand_chrom = random.choice(offspring_crossover)
        #     row = 1
        #     for idx, value in enumerate(rand_chrom):
        #         width_val += value.width
        #         if idx % 3 == 0:
        #             if width_val > self.container_info[0]:
        #                 overwidth_arr.append(row)
        #             width_val = 0
        #             row += 1
        #     for x in range(len(overwidth_arr)):
        #         rand_chrom[overwidth_arr[x]] = StacksMaker.stack_zero
        #     offspring_mutation.append(rand_chrom)

        # mutate_value_one = None
        # mutate_value_two = None
        # mutate_value_three = None
        # for _ in range(10):
        #     mutate_chrom = random.choice(offspring_crossover)
        #     # if self.find_sum_area(mutate_chrom) > (self.container_info[0] * self.container_info[1]):
        #     #     mutate_chrom[random.randint(0, 32)] = StacksMaker.stack_zero
        #     #     mutate_chrom[random.randint(0, 32)] = StacksMaker.stack_zero
        #     #     mutate_chrom[random.randint(0, 32)] = StacksMaker.stack_zero
        #     # else :
        #     width_arr = list()
        #     for idx, value in enumerate(mutate_chrom):
        #         width_arr.append(value.width)
        #         if idx % 3 == 0:
        #             if sum(width_arr) > self.container_info[0]:
        #                 random_stack = random.choice(stacks)
        #                 if sum([random_stack.width, width_arr[1], width_arr[2]]) < self.container_info[0]:
        #                     mutate_chrom[idx-2] = random_stack
        #                 elif sum([width_arr[0], random_stack.width, width_arr[2]]) < self.container_info[0]:
        #                     mutate_chrom[idx-1] = random_stack
        #                 elif sum([width_arr[0], width_arr[1], random_stack.width]) < self.container_info[0]:
        #                     mutate_chrom[idx] = random_stack
        #                 else :
        #                     mutate_chrom[random.randint(idx-3, idx)] = StacksMaker.stack_zero
        #             width_arr.clear()
                            
                # if self.find_section_weight_area(0, mutate_chrom)[0] > (self.find_sum_area(mutate_chrom) / 3):
                #     diff = self.find_section_weight_area(0, mutate_chrom)[0] - self.find_sum_area(mutate_chrom) / 3
                #     if diff > 3.3633:
                #         for j in mutate_chrom:
                #             if j.name == 106 or j.name == 207:
                #                 for x in self.stacks:
                #                     if x.name == 101 or x.name == 102 or x.name == 201 or x.name == 202:
                #                         mutate_chrom[mutate_value_one] = x
            #     mutate_value_one = random.choice(self.stacks)
            #     mutate_value_two = random.choice(self.stacks)
            #     mutate_value_three = random.choice(self.stacks)
            # mutate_pos_one = random.randint(0, 10)
            # mutate_pos_two = random.randint(11, 21)
            # mutate_pos_three = random.randint(22, 32)
            # mutate_chrom[mutate_pos_one] = mutate_value_one
            # mutate_chrom[mutate_pos_two] = mutate_value_two
            # mutate_chrom[mutate_pos_three] = mutate_value_three

            # offspring_mutation.append(mutate_chrom)
        return offspring_mutation
    def find_sum_area(self, chromosome):
        area = 0
        for j in chromosome:
            area += j.area
        return area
    def find_section_weight_area(self, section, chrom):
        area = 0
        weight = 0
        if section == 0:
            min_range = 0
            max_range = 10
        elif section == 1:
            min_range = 11
            max_range = 21
        elif section == 2:
            min_range = 22
            max_range = 32
        for j in chrom[min_range, max_range]:
            area += j.area
            for k, v in j.weight_each.items():
                for l in v:
                    weight += l
        return area, weight
    def check_width(self, chrom):
        over = False
        width = 0
        for idx, value in enumerate(chrom):
            if idx % 3 == 0:
                if width > self.container_info[0]:
                    over = True
                width = True
        return over
    def execute(self):
        initial_population = self.initial_population_generator()
        population = initial_population
        score_stack = 0
        prev_score = 0
        x = 0
        limit = 5000
        while True:
            clear_output(wait=True)
            print(f'Runing generation {x+1} ...')
            chromosome_scores = self.fitness_calculation(population, self.container_id)
            print(f"Highest chromosome score = {max(chromosome_scores)}")
            if self.find_sum_area(population[chromosome_scores.index(max(chromosome_scores))]) > (self.container_info[0] * self.container_info[1]):
                if self.check_width(population[chromosome_scores.index(max(chromosome_scores))]):
                    break
            if x > 50000:
                break
            # if x > 1000:
            #     break
            # if round(max(chromosome_scores), 1) == 1.0:
            #     break
            # if max(chromosome_scores) >= 1.0:
            #     break
            if round(prev_score, 2) == round(max(chromosome_scores), 2):
                score_stack += 1
                if score_stack == limit:
                    break
            else :
                score_stack = 0

            offspring_selection = self.selection(copy.deepcopy(population), chromosome_scores)
            offspring_crossover = self.crossover(copy.deepcopy(offspring_selection))
            offspring_mutation = self.mutation(copy.deepcopy(offspring_crossover), self.stacks)
            new_population = offspring_selection + offspring_crossover + offspring_mutation
            population = copy.deepcopy(new_population)
            prev_score = max(chromosome_scores)
            x += 1
        return population, chromosome_scores

In [18]:
container_id = 'c5'
pattern_generator = GeneticAlgorithm(stacks, container_id)
population, chromosome_scores = pattern_generator.execute()

KeyError: 'c5'

In [ ]:
def get_area(chrom):
    area = 0
    for x in chrom:
        area += x.area
    return area
def to_json(chrom, container_type):
    dict = {
        "Data" : {
            "Container_Type": container_type,
            "Area" : get_area(chrom),
            "Container" : [
                [],
                [],
                [],
                [],
                [],
                [],
                [],
                [],
                [],
                [],
                []
        ]
        }
    }
    row_cnt = -1
    sum_weight = 0
    weight_sections = []
    sum_width = 0
    for x, j in enumerate(chrom):
        if x % 3 == 0:
            if sum_width > parcels.Info.get_container_info(container_id)[0]:
                print(f"Row#{row_cnt}=> overwidth")
            sum_width = 0
            row_cnt += 1
        dict['Data']['Container'][row_cnt].append(j.name)
        for y, z in j.weight_each.items():
            for o in z:
                sum_weight += o
        if x == 10 or x == 21 or x == 32:
            weight_sections.append(sum_weight)
            sum_weight = 0
        sum_width += j.width
    print(sum(weight_sections))
    # for i in weight_sections:
    #     print(round((i / sum(weight_sections) * 100), 2))
    dict['Data']['Weight'] = sum(weight_sections)
    return dict, weight_sections
    

In [ ]:
to_json(population[chromosome_scores.index(max(chromosome_scores))], container_id)

2140


({'Data': {'Container_Type': 'c5',
   'Area': 27.238399999999988,
   'Container': [[101, 201, 101],
    [101, 101, 102],
    [201, 101, 101],
    [101, 102, 103],
    [102, 101, 101],
    [101, 101, 101],
    [101, 101, 201],
    [101, 102, 101],
    [0, 101, 101],
    [101, 102, 101],
    [101, 102, 101]],
   'Weight': 2140}},
 [715, 767, 658])

In [ ]:
max(chromosome_scores)

-0.24276466029449884